In [1]:
import pandas as pd
import numpy as np
import re

In [4]:
df = pd.read_csv('../data/obf_data.csv')
df

,payload,length,is_obf,label
0,c/ caridad s/n OR 1123-398=1394-363 OR 960-75=...,233,1,0
1,"campello , el AND 167-158=560-551 OR 772-763=8...",294,1,0
2,41159-975 AND 370-366=609-605 AND 987-979=624-...,748,1,0
3,1442431887503494-164 AND 649-648=369-368,40,1,0
4,nue836-799 OR 1610-645=2127-248 AND 931-927=88...,581,1,0
...,...,...,...,...
30151,"170-169' and extractvalue 8981-999 , concat 37...",563,1,1
30152,"409-408"" order by 810-809-- OR 954-431=1452-17...",959,1,1
30153,380-379' procedure analyse extractvalue 6547-7...,977,1,1
30154,875-874 rlike select * from select sleep 276-2...,100,1,1


In [5]:
def clean_string(query):
    return re.sub('\s+', ' ', query.replace('(', ' ').replace(')', ' ').replace(',', ' , ')).strip()

In [6]:
df['payload'] = df['payload'].apply(clean_string)
benign = df.loc[df.label == 0]
malicious = df.loc[df.label == 1]

In [12]:
data = benign.append(malicious.iloc[0])
data

,payload,length,is_obf,label
0,c/ caridad s/n OR 1123-398=1394-363 OR 960-75=...,233,1,0
1,"campello , el AND 167-158=560-551 OR 772-763=8...",294,1,0
2,41159-975 AND 370-366=609-605 AND 987-979=624-...,748,1,0
3,1442431887503494-164 AND 649-648=369-368,40,1,0
4,nue836-799 OR 1610-645=2127-248 AND 931-927=88...,581,1,0
...,...,...,...,...
26481,esdrs OR 591-533=1871-823 AND 457-451=501-495 ...,593,1,0
26482,violy belcastro OR 483-146=1662-906 OR 710-114...,435,1,0
26483,celeste_bryon@skullride.ki AND 547-547=941-941...,323,1,0
26484,98970543w AND 4=4 AND 7=7 OR 857=1191 OR 663=1...,270,1,0


In [20]:
from sklearn.feature_extraction.text import CountVectorizer
from scipy import sparse

count_vect = CountVectorizer(analyzer='word', ngram_range=(3,3))
raw_counts = count_vect.fit_transform(data['payload'])

# Append metadata to sparse matrix that will be used to train the classifier
num_feats = data[['length']].values
train_data = sparse.hstack((raw_counts, num_feats))

In [21]:
from sklearn.ensemble import IsolationForest

score = IsolationForest(random_state = 42).fit(train_data).decision_function(train_data)
ranked_index = np.argsort(score) 
malicious_rank = ranked_index.tolist().index(19304)

In [17]:
malicious_rank

19304

In [19]:
import warnings
warnings.filterwarnings('ignore')

samples = 1

dataset = [benign.append(malicious.iloc[i]) for i in range(samples)]
count_vect = CountVectorizer(analyzer='word', ngram_range=(3,3))
raw_counts = [count_vect.fit_transform(data['payload']) for data in dataset]

# Append metadata to sparse matrix that will be used to train the classifier
lengths = [data[['length']].values for data in dataset]
train_data = [sparse.hstack((raw_counts[i], lengths[i])) for i in range(samples)]
scores = [IsolationForest(random_state = 42).fit(data).decision_function(data) for data in train_data]
ranked_indexes = [np.argsort(score) for score in scores]
malicious_rank = [l.tolist().index(19304) for l in ranked_indexes]
print('Anamoly rank of the malicious datapoint:')
for i in range(15):
    print('datapoint %d: %d' % (i, malicious_rank[i]))

print('Average rank = %d' % (sum(malicious_rank)/len(malicious_rank)))

KeyboardInterrupt: 

In [ ]:
# import warnings
# warnings.filterwarnings('ignore')

# samples = 100

# dataset = [benign.append(malicious.iloc[i]) for i in range(samples)]
# count_vect = CountVectorizer(analyzer='word', ngram_range=(3,3))
# raw_counts = [count_vect.fit_transform(data['payload']) for data in dataset]

# # Append metadata to sparse matrix that will be used to train the classifier
# lengths = [data[['length']].values for data in dataset]
# train_data = [sparse.hstack((raw_counts[i], lengths[i])) for i in range(samples)]
# scores = [IsolationForest(random_state = 42).fit(data).decision_function(data) for data in train_data]
# ranked_indexes = [np.argsort(score) for score in scores]
# malicious_rank = [l.tolist().index(19304) for l in ranked_indexes]
# print('Anamoly rank of the malicious datapoint:')
# for i in range(15):
#     print('datapoint %d: %d' % (i, malicious_rank[i]))

# print('Average rank = %d' % (sum(malicious_rank)/len(malicious_rank)))

In [ ]:
np.sort(malicious_rank)